<a href="https://colab.research.google.com/github/danielpatrickhug/Aquarium/blob/main/Knowledge_Distilliation_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge Distillation
Reference: [Hinton et al. (2015)](https://arxiv.org/abs/1503.02531)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import os
import time

## Teacher Setup

In [37]:
batch_size_teacher = 128
test_batch_size_teacher= 1000
epochs_teacher = 6
epochs_student = 3
lr_teacher =0.01
momentum_teacher = 0.9
seed = 42
log_interval = 10
no_cuda = False

In [3]:
cuda = not no_cuda and torch.cuda.is_available()
use_cuda= True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

In [4]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [6]:
train_kwargs = {'batch_size': batch_size_teacher}
test_kwargs = {'batch_size': test_batch_size_teacher}

In [7]:
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

In [8]:
%%capture
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

## Teacher Model

In [10]:
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.fc1 = nn.Linear(784, 1200)
        self.fc2 = nn.Linear(1200, 1200)
        self.fc3 = nn.Linear(1200, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.8)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.8)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [20]:
def teacher_train(epoch, model):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        

In [23]:
def teacher_train_evaluate(model):
    model.eval()
    train_loss = 0
    correct = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        train_loss += F.cross_entropy(output, target).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))

In [25]:
def teacher_test(model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.cross_entropy(output, target).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Teacher Training

In [26]:
teacher_model = Teacher()
teacher_model.to(device)
optimizer = optim.SGD(teacher_model.parameters(), lr=lr_teacher, momentum=0.5,
                      weight_decay=5e-4)
for epoch in range(1, epochs_teacher + 1):
    teacher_train(epoch, teacher_model)
    teacher_train_evaluate(teacher_model)
    teacher_test(teacher_model)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.400108
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.364313
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.309432
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.110107
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.063204
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.042824
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.826525
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.827687
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.638777
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.509249
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.415824
Train Epoch: 1 [14080/60000 (23%)]	Loss: 1.427236
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.304447
Train Epoch: 1 [16640/60000 (28%)]	Loss: 1.256538
Train Epoch: 1 [17920/60000 (30%)]	Loss: 1.241844
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.093959
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.217756
Train Epoch: 1 [21760/60000 (36%)]	Loss: 1.104575
Train Epoch: 1 [23040/60000 (38%)]	Loss: 1.002648
Train Epoch: 1 [24320/60000 (41%)]	Loss: 1.032651
Train Epoch: 1 [

## Student Model

In [34]:
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 800)
        self.fc2 = nn.Linear(800, 800)
        self.fc3 = nn.Linear(800, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Distillation

In [29]:
def distillation(y, labels, teacher_scores, T, alpha):
    return nn.KLDivLoss()(F.log_softmax(y/T), F.softmax(teacher_scores/T)) * (T*T * 2.0 * alpha) + F.cross_entropy(y, labels) * (1. - alpha)

In [30]:
def train(epoch, model, loss_fn):
    model.train()
    teacher_model.eval()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        teacher_output = teacher_model(data)
        teacher_output = teacher_output.detach()
        # teacher_output = Variable(teacher_output.data, requires_grad=False) #alternative approach to load teacher_output
        loss = loss_fn(output, target, teacher_output, T=20.0, alpha=0.7)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [31]:
def train_evaluate(model):
    model.eval()
    train_loss = 0
    correct = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        train_loss += F.cross_entropy(output, target).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))

In [32]:
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # test_loss += F.cross_entropy(output, target).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [35]:
model = Student()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

## Student Training

In [38]:
for epoch in range(1, epochs_student + 1):
    train(epoch, model, loss_fn=distillation)
    train_evaluate(model)
    test(model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.121444
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.985574
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.876781
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.898033
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.732158
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.849338
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.687862
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.801792
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.597332
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.562558
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.405815
Train Epoch: 1 [14080/60000 (23%)]	Loss: 1.275338
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.132553
Train Epoch: 1 [16640/60000 (28%)]	Loss: 1.112868
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.886485
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.889022
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.794079
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.788738
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.732066
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.736556
Train Epoch: 1 [